### Task1


Provide the table DDL for trivago.advertiser_hotels
1. Create a table first
```
            CREATE TABLE IF NOT EXISTS advertiser_hotels_ddl (
            hotel_id string,
            hotel_name string,)
            COMMENT 'DDL table'
            ROW FORMAT DELIMITED
            FIELDS TERMINATED BY ',';
```
2. Load data into the table
```
    LOAD DATA INPATH '/input_files/bedandBLD_master_list_20200203.csv' INTO TABLE advertiser_hotels_ddl
    LOAD DATA INPATH '/input_files/lookandbook_hotel_list_20200203.csv' INTO TABLE advertiser_hotels_ddl
```
Provide a list of commands needed or a step-by-step guide to load the data into HDFS
1. Starting HDFS
```
    hadoop namenode -format
```
2. Create an input directory
```
    hadoop fs -mkdir /user/solution/task1/
```
3. Put files into the folder
```
    hadoop fs -put /input_files/lookandbook_hotel_list_20200203.csv /input_files/bedandBLD_master_list_20200203.csv /user/solution/task1/
```
4. In hive, 
```
    hive> CREATE TABLE IF NOT EXISTS load_advertiser_hotels (
            hotel_id string,
            hotel_name string,)
            ROW FORMAT DELIMITED
            FIELDS TERMINATED BY ',';

    hive> LOAD DATA INPATH '/input_files/bedandBLD_master_list_20200203.csv' INTO TABLE load_advertiser_hotels
          LOAD DATA INPATH '/input_files/lookandbook_hotel_list_20200203.csv' INTO TABLE load_advertiser_hotels
``` 

### Task2
 data quality check


In [5]:
import os
import re
import csv
from os import listdir
from os.path import isfile, join

path = os.getcwd()+"/input_files/"

ALLOWED_EXTENSTIONS = {".csv"}
ALLOWED_DELIMITER =[',']

def is_file_name_format_OK(filelist):
    """The file name format check

    Args:
      filelist: a file list in the directory

    Returns:
      True if the format is OK
    """

    for filename in filelist:
        extension = os.path.splitext(filename)[1]
        if extension not in ALLOWED_EXTENSTIONS:
          print("{} is not a valid file name format".format(filename))
          return False
    return True
            

def is_file_empty(filelist,dirpath):
    """The empty file check

    Args:
      filelist:a  file list in the directory

    Returns:
      True if the file empty 
    """
    for filename in filelist:
      filepath = os.path.join(dirpath,filename)
      if (os.path.exists(filepath) and os.stat(filepath).st_size == 0):
        print("{} does not exist or is empty.".format(filename))
        return True
    return False


def has_file_header(csvfile):
  """The file's header check

    Args:
      csvfile object

    Returns:
      True if the file has header 
  """
  sampledata =csvfile.read(1024)
  return csv.Sniffer().has_header(sampledata)

def is_wrong_delimiter(csvfile):
  """The file's delimiter check

    Args:
      csvfile object

    Returns:
      True if the file is wrong delimiter 
  """
  is_wrong_delimiter = False
  try:
    dialect = csv.Sniffer().sniff(csvfile.readline(), ALLOWED_DELIMITER)
  except:
    is_wrong_delimiter = True
  return is_wrong_delimiter

def has_extra_comma(csvfile):
  """The file's extra comma check

    Args:
      csvfile object

    Returns:
      True if the file has extra comma
  """
  csvreader = csv.reader(csvfile, delimiter = ",")
  for linenum,fieldlist in enumerate(csvreader):
    if '' in set(fieldlist):
      print("Line.{} has the extra comma.".format(linenum+1))
      return True
  return False


def has_specialchar(csvfile):
  """The file's special character check

    Args:
      csvfile object

    Returns:
      True if the file has special character
  """
  csvreader = csv.reader(csvfile, delimiter = ",")
  for fieldlist in csvreader:
    for field in fieldlist:
        if re.compile('[^ 0-9a-zA-Z_-]+').search(field):
          print("{} contains special character.".format(field))
          return True
  return False


def data_quality_check(path):

  filenames = [f for f in listdir(path) if isfile(join(path, f))]

  # for the first two check, I check all the files because it saves time by not including "with open" operation
  if not is_file_name_format_OK(filenames):
    print("Check the file format!")
    return
  
  if is_file_empty(filenames,path):
    print("Check the file empty!")
    return

  for filename in filenames:
    filepath = os.path.join(path,filename)
    with open(filepath, 'r', encoding='UTF-8') as csvfile:
      #check the header
      if not has_file_header(csvfile):
        print("{} does not have header.".format(filename))
        return 
      else:
        #check the delimiter
        csvfile.seek(0)
        if is_wrong_delimiter(csvfile):
          print("{} is wrong delimiter.".format(filename))
          return
        else:
          #check extra comma
          csvfile.seek(0)
          if has_extra_comma(csvfile):
            print("{} has extra comma.".format(filename))
            return
          else:
            #check special character
            csvfile.seek(0)
            if has_specialchar(csvfile):
              print("{} has special character.".format(filename))
              return
  print ("data quality check OK.")

data_quality_check(path) 

data quality check OK.


### Deal with the analytics issues

#### Design proposal
Create a fact table called hotel.
A dimention table called hotel_info and a dimention table called time.\
In hotel table, the id is sequencial increament id which is primary key 
and make hotel_id and start_time as the foreign keys for the hotel_info table and time table respecitively.\
In this case, hotel_id is unique and not null.\
When inserting values into hotel_info table, make the hotel_name Distict to resolve duplicate issue.\
Extract the start_time value to create the respective fields in time table.\
Join the tables, can let analysts track the time for a hotel.

Fact table:

    hotel table
    - id, Primary Key, Int
    - hotel_id, Foreign Key, Int
    - start_time, Foreign Key, Timestamp

Dimention table:

    hotel_info table
    - hotel_id, Primary Key, Int
    - hotel_name,Varchar
    - active,Boolean
    - removedbefore,Boolean

    time table
    - start_time,Primary Key, Timestamp
    - hour	Int
    - day	Int
    - week	Int
    - month	Int
    - year	Int





